Nystro🐈Nyan Nyora [WIP?]


The work is based on
* idea of LoRA (take little parameters to represent big matrix)
* idea of Nyströmformer and Nyström approximation (take 1 matrix, beat it into smaller parts)


(PLACEHOLDER FOR A KITTEN OR A CATGIRL)

In [2]:
# imports,  let's skip this part, it's not my favorite
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

In [3]:
# Settings
from pathlib import Path  #(only place where it will be used )

MODEL_PATH = Path("~/models/ahxt_LiteLlama-460M-1T").expanduser()
DTYPE = torch.bfloat16
ATTN_IMPL="flash_attention_2"
DEVICE = "cuda"
RANK=32
BATCH_SIZE=5
N_CTX=640
DATASET = "iohadrubin/wikitext-103-raw-v1"

SPOILER: In the end I've found that LiteLlama tokenizer eats double quote '"'...

(https://huggingface.co/ahxt/LiteLlama-460M-1T/discussions/9#659f6f894c074ce5e4e9532c)

... as it thinks it's a [PAD], so results are questionable, need to check on other model

In [4]:
# Let's build Nystronyan block that uses less than (NxN) parameters to be 
# To read inspration, check https://arxiv.org/pdf/2102.03902.pdf, however we need only a single formula from there
# as we don't care that much of finer details such as "optimization"
class NystroNyan(nn.Module):
    def __init__(self, in_features: int, out_features: int, r: int):
        
        super().__init__()
        # mirror nn.Linear reports
        self.in_features = in_features
        self.out_features = out_features
        self.rank = r

        # big matrix n x n, split into four parts
        #   
        #  (r r)          (r, n_col-r) 
        #    A                  B
        #    F                  C
        #  (n_row-r r)  (n_row-r n_col-r)
        # our part of interest  is C.

        self.A = nn.Parameter(torch.rand(r, r))
        self.B = nn.Parameter(torch.rand(r, out_features-r))
        self.F = nn.Parameter(torch.rand(in_features-r, r))
        
    @property
    def A_(self):
        # pinverse doesn't support bf16
        return self.A.float().pinverse().to(dtype=self.A.dtype)

    def forward(self, x):
        # we need Moore-Penrose inverse
        A_ = self.A_

        AB = torch.cat((self.A, self.B), -1)  # AB upper half
        AF = torch.cat((self.A, self.F), 0)   # AF left half
        
        reconsturcted_matrix = AF @ A_ @ AB

        return x @ reconsturcted_matrix



In [20]:
# Sanity check
def model_numel(m, grad_only=False):
    return sum(p.numel() for p in m.parameters() if p.requires_grad or not grad_only)
    
def sanity_check(n=4096, r=32):
    print(f"\n({n=}, {r=})")
    print("NystroNyan:\t", model_numel(NystroNyan(n, n, r)))
    print("LoRA:\t", torch.zeros(2, n, r).numel())

sanity_check(r=8)
sanity_check(r=16)
sanity_check(r=32)
# We are more parameter efficient. Nyaa.


(n=4096, r=8)
NystroNyan:	 65472
LoRA:	 65536

(n=4096, r=16)
NystroNyan:	 130816
LoRA:	 131072

(n=4096, r=32)
NystroNyan:	 261120
LoRA:	 262144


In [6]:
# Loading a model
def reset_model():
    global model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=DTYPE,
        attn_implementation=ATTN_IMPL
    ).to(device=DEVICE)
    return model
reset_model()    

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50304, 1024, padding_idx=0)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1024, out_features=128, bias=False)
          (v_proj): Linear(in_features=1024, out_features=128, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (up_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )


In [7]:
# Loading a tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token_id is None:    
    tokenizer.pad_token_id = tokenizer.unk_token_id
tokenizer_path = MODEL_PATH

In [8]:
# For data loading
def tokenize_text(samples, tokenizer):
    return {"input_ids": tokenizer(samples["text"]).input_ids}

In [9]:
# for data loading
def get_dl(batch_size, split="train"):
    ds = datasets.load_dataset(DATASET)
    ds = ds[split]
    assert "text" in ds.column_names
    ds = ds.map(
        tokenize_text,
        remove_columns=ds.column_names,
        fn_kwargs={"tokenizer": tokenizer},
        batched=True)
    collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        collate_fn=collator,
        shuffle=False)
    return dl

import functools

# We cache result to omit stupid messages like "Found cached dataset parquet"
@functools.cache
def get_dl_cached(batch_size, split="train"):
    return get_dl(batch_size, split)

def iter_batch(batch: dict, n_ctx: int):
    for start in range(0, batch.input_ids.shape[1], n_ctx):
        part = {k: v[:, start:start+n_ctx].to(DEVICE) for k,v in batch.items()}
        part["progress"] = (start, batch.input_ids.shape[1])
        yield part

In [10]:
# Now we need a module for injecting our cute kitten
class NyanInjector(nn.Module):
    def __init__(self, base: nn.Linear, r: int) -> None:
        super().__init__()
        assert isinstance(base, nn.Linear)
        self.base = base
        self.delta = NystroNyan(base.in_features, base.out_features, r)
        for p in self.delta.parameters():
            p.data *= model.config.initializer_range
        
    def forward(self, x):
        y = self.base(x)
        y = y + self.delta(x)
        return y



In [ ]:
# Freeze everything ❄
model.requires_grad_(False)

# Hot catgirls go in 🔥
for layer in model.model.layers:
    attn = layer.self_attn
    attn.q_proj = NyanInjector(attn.q_proj, r=RANK).to(dtype=DTYPE, device=DEVICE)
    attn.v_proj = NyanInjector(attn.v_proj, r=RANK).to(dtype=DTYPE, device=DEVICE)

parms_to_train = model_numel(model, grad_only=True)
parms_total = model_numel(model, grad_only=False) 
print(f"#parms: {parms_total}, trainable: {parms_to_train} ({parms_to_train/parms_total:%})")

# In PEFT LoRA: trainable params: 2457600 || all params: 464143360 || trainable%: 0.529491577774591

In [11]:
dl = get_dl(batch_size=BATCH_SIZE)

url='https://huggingface.co/datasets/iohadrubin/wikitext-103-raw-v1/resolve/main/dataset_infos.json'
cached_url='https://huggingface.co/datasets/iohadrubin/wikitext-103-raw-v1/resolve/main/dataset_infos.json', cache_path='/home/fella/.cache/huggingface/datasets/downloads/fd88374720898676c1aac99c1637230812afd06875ab0daf8df9c2e940e5eb56.ef59c73aaa8d4724466db27d4c6ee30e773d72f4a9ad4791ce2a85c70acc8e72'
url='https://huggingface.co/datasets/iohadrubin/wikitext-103-raw-v1/resolve/main/README.md'


Found cached dataset parquet (/home/fella/.cache/huggingface/datasets/iohadrubin___parquet/iohadrubin--wikitext-103-raw-v1-a0b8bc67accf87c3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Not connected, checking cache `/home/fella/.cache/huggingface/datasets/downloads/b03c45f13f201de25b7a5863a2ac945f80be7ca141c039f519075e10bda67e2c for https://huggingface.co/datasets/iohadrubin/wikitext-103-raw-v1/resolve/main/README.md`


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/fella/.cache/huggingface/datasets/iohadrubin___parquet/iohadrubin--wikitext-103-raw-v1-a0b8bc67accf87c3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f17f8ff98091523f.arrow


In [18]:
opt = torch.optim.AdamW(model.parameters(), fused=True)

for ib, b in enumerate(bar := tqdm(dl)):
    if ib < 101:
        continue
    for mb in iter_batch(b, N_CTX):
        (cur, total) = mb.pop("progress")
        loss = model(**mb).loss
        loss.backward()
        opt.step()
        opt.zero_grad()
        bar.set_description(f'L:{loss.item():.4} P:{cur/total:%}')


  0%|          | 0/5914 [00:00<?, ?it/s]

In [ ]:
# unload nyora
for layer in model.model.layers:
    attn = layer.self_attn
    assert isinstance(attn.q_proj, NyanInjector)
    attn.q_proj = attn.q_proj.base
    attn.v_proj = attn.v_proj.base
    

In [12]:
from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                         inference_mode=False, r=RANK, 
                         lora_alpha=32, 
                         lora_dropout=0.1, 
                         target_modules=['q_proj', 'v_proj'])
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2457600 || all params: 464143360 || trainable%: 0.529491577774591


In [23]:
@torch.no_grad()
def run_valid():
    dl = get_dl_cached(batch_size=1, split="validation")
    losses = []
    model.eval()
    for _, b in enumerate(bar := tqdm(dl)):
        for mb in iter_batch(b, N_CTX):
            (cur, total) = mb.pop("progress")
            loss = model(**mb).loss.item()
            bar.set_description(f'L:{loss:.4} P:{cur/total:%}')
            losses.append(loss)
    model.train()
    return torch.tensor(losses).mean().item()
            

run_valid()
# Vanila: 3.132427930831909

# Nyora  v 
#  100b5: 3.131727695465088
# 1295b5: 3.033968448638916
# 1950b5: 3.055903434753418
# 2450b5: 3.0657265186309814
# 5914b5: 3.0219664573669434

# Lora
# 0(no train): 3.132427930831909
#  101b5: 3.0502851009368896
# 5914b5: 3.025418281555176



  0%|          | 0/60 [00:00<?, ?it/s]

3.025418281555176

In [57]:
for x in get_dl_cached(split="validation", batch_size=1):
    print(model.config)

    break

LlamaConfig {
  "_name_or_path": "/home/fella/models/ahxt_LiteLlama-460M-1T",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layernorm_epsilon": 1e-05,
  "max_position_embeddings": 1024,
  "model_type": "llama",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.2",
  "use_cache": true,
  "vocab_size": 50304
}



In [101]:
def sanity_prompt(p):
    x = tokenizer(p, return_tensors="pt").to(DEVICE)
    #del x["token_type_ids"]
    x = model.generate(**x, max_new_tokens=20, do_sample=True).ravel()
    x2 = tokenizer.decode(x)
    return x,x2
x1, x2 = sanity_prompt("Smiling, waving to the rhythm of the music, said to him only word:")
for x in x1:
    print(x, f'>'+tokenizer.decode(x)+'<')
print(x2)

tensor(7556, device='cuda:0') >Sm<
tensor(4386, device='cuda:0') >iling<
tensor(11, device='cuda:0') >,<
tensor(25849, device='cuda:0') > waving<
tensor(284, device='cuda:0') > to<
tensor(262, device='cuda:0') > the<
tensor(18662, device='cuda:0') > rhythm<
tensor(286, device='cuda:0') > of<
tensor(262, device='cuda:0') > the<
tensor(2647, device='cuda:0') > music<
tensor(11, device='cuda:0') >,<
tensor(531, device='cuda:0') > said<
tensor(284, device='cuda:0') > to<
tensor(683, device='cuda:0') > him<
tensor(691, device='cuda:0') > only<
tensor(1573, device='cuda:0') > word<
tensor(25, device='cuda:0') >:<
tensor(198, device='cuda:0') >
<
tensor(198, device='cuda:0') >
<
tensor(1, device='cuda:0') >"<
tensor(1858, device='cuda:0') >There<
tensor(318, device='cuda:0') > is<
tensor(257, device='cuda:0') > a<
tensor(2415, device='cuda:0') > woman<
tensor(994, device='cuda:0') > here<
tensor(9975, device='cuda:0') > tonight<
tensor(553, device='cuda:0') >,"<
tensor(339, device='cuda:0') >

In [15]:
model.model.model.layers[1].self_attn.q_proj

Linear(
  in_features=1024, out_features=1024, bias=False
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.1, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=32, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=32, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
)

In [127]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token_id = 0
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
features = tokenizer('hello! hello!', return_tensors="pt")
collator([features])


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[[31373,     0, 23748,     0]]]), 'attention_mask': tensor([[[1, 1, 1, 1]]]), 'labels': tensor([[[31373,  -100, 23748,  -100]]])}

In [130]:
tokenizer2("!")

{'input_ids': [0], 'attention_mask': [1]}